# Focus su Runnables, "pipe" e Chain

In [ ]:
#pip install -U ollama
#pip install -qU langchain-ollama

In [1]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableParallel
from langchain_ollama import ChatOllama

In [ ]:
#llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=os.getenv("openai_key"))

 
model = ChatOllama(model="llama3.2", temperature=0.)


In [2]:
from langchain.chat_models import init_chat_model

model = init_chat_model("ollama:llama3.2") 

In [3]:
prompt = ChatPromptTemplate.from_template("descrivi correttamente {topic} in massimo 10 parole")

In [4]:
chain = prompt | model | StrOutputParser()

In [5]:
# invocazione standard

query = "sognare"

chain.invoke(query)

'Sognare è stato durante il sonno con ricordi vividi e intensi.'

In [6]:
# esplicitazione del parametro di input

query = {"topic": "sognare"}

chain.invoke(query)

'Sognare significa avere immagini significative durante il sonno.'

In [7]:
analysis_prompt = ChatPromptTemplate.from_template("è una descrizione corretta e breve?\n{topic}")

composed_chain = {"topic": chain} | analysis_prompt | model | StrOutputParser()

In [8]:
composed_chain.invoke(query)

'Sì, la descrizione è corretta e breve.\n\nLa frase "Sognare è stato sperimentato da tutti gli esseri umani" esprime un concetto fondamentale della psicologia e della cognizione umana. In effetti, sognare è un fenomeno universale che affetta tutti gli esseri umani, indipendentemente dalla cultura, dall\'età o dalla personalità.\n\nLa descrizione è anche breve e concisa, rendendo facile la comprensione del concetto in poche parole.'

In [9]:
# ulteriore sintassi per la catena composta

composed_chain = (
    chain
    | (lambda input: {"topic": input + "\n"})   # superfluo, potrebbe essere utile per trasformare l'output della prima catena prima di sottometterlo alla seconda
    | analysis_prompt
    | model
    | StrOutputParser()
)

In [10]:
composed_chain.invoke(query)

'No, la descrizione non è del tutto corretta.\n\nSognare è quando ci immaginiamo qualcosa mentre dormiamo. La descrizione più precisa sarebbe:\n\n"Sognare è quando la mente si occupa di immagini, pensieri e emozioni mentre siamo in stato di sonno, creando scene virtuale che possono essere molto reali per noi."\n\nLa frase originale manca l\'importante particolare "mente" e aggiunge un po\' di informazione su cosa accade durante il sogno.'

In [11]:
composed_chain = (
    chain
    .pipe(lambda input: {"topic": input + "\n"})
    .pipe(analysis_prompt)
    .pipe(model)
    .pipe(StrOutputParser())
)

composed_chain.invoke(query)

"No, la descrizione non è esatta.\n\nSognare è generalmente considerato uno stato di coscienza ipnagogico, caratterizzato da un'involuzione della consapevolezza e della attenzione. In questo stato, le linee di confine tra il mondo esterno e l'interiore sono meno chiare rispetto a uno stato normale di veglia.\n\nUn estado di coscienza profondo e inconscio è più vicino a uno stato di meditazione o di trance, caratterizzato da un'estensione della consapevolezza verso l'inconscio e una riduzione delle linee di confine tra il soggetto e il mondo esterno."

### Runnable

`Runnable` è un'interfaccia che rappresenta elementi eseguibili all'interno di una pipeline LangChain.

Gli oggetti Runnable implementano metodi come `invoke()`, `stream()` e `batch()`, che permettono di eseguire il processo in modalità sincrona, asincrona o in batch.

`RunnableLambda` consente di trasformare una funzione Python in un oggetto `Runnable`, rendendola compatibile con il framework.

In [12]:
composed_chain_with_runnable_lambda = (
    chain
    | RunnableLambda(lambda input: {"topic": input})  # superfluo - prendiamo l'input e lo trasformiamo in un dizionario, prima di passarlo a "analysis_prompt"
    | analysis_prompt
    | model
    | StrOutputParser()
)

composed_chain_with_runnable_lambda.invoke(query)

'Sì, la frase "Sognare è stato più vivo che non pensare" è una descrizione breve e corretta.\n\nLa frase suggerisce che il sogno o l\'immaginazione può essere più vissuto e profondamente esperienziato rispetto alla semplice riflessione o al pensiero. In altre parole, il sogno può offrire un\'esperienza più intensa e emozionale rispetto alla meramente astrazione.\n\nLa frase ha anche un tono poetico e filosofico, che suggerisce una certa profondità e complessità nel suo significato.'

In [13]:
sequence_chain = (
    {"topic": RunnablePassthrough()}  # superfluo - fa passare l'input fornito, trasformandolo in Runnable, associa quindi l'input alla chiave "topic", fornita in input al "prompt"
    | prompt
    | model
    | StrOutputParser()
    | {"topic": RunnablePassthrough()}
    | analysis_prompt
    | model
    | StrOutputParser()
)

sequence_chain.invoke("sognare")

'Certo, la descrizione è corretta e breve!\n\n"Sognare è un\'esperienza soggettiva ed emotiva" è una frase che riassume efficacemente il concetto di sogno come esperienza personale e sentimentale. La parola "soggettiva" sottolinea che i sogni sono esperienze unique e personali per ciascuno, mentre la parola "emotiva" evidenzia l\'aspetto emotivo e sentito dei sogni.\n\nLa descrizione è anche breve e concisa, facile da capire e comprenderere.'

In [14]:
sequence_chain = (
    prompt
    | model
    | StrOutputParser()
    | analysis_prompt
    | model
    | StrOutputParser()
)

sequence_chain.invoke("sognare")

'Certo, la frase "Sognare è stato dal cervello, mentre dormiamo" è una descrizione breve ed efficace della funzione del sonno e dell\'attività cerebrale durante il sogno.\n\nLa frase è anche chiara e facile da capire, e trasmette in modo conciso l\'idea che i sogni siano prodotti dall\'attività del cervello durante lo stato di dormienza.'

### RunnableParallel

Grazie agli elementi Runnable è possibile eseguire più elementi in parallelo utilizzando `RunnableParallel`, potendo così creare catene anche molto complesse e sfruttando il parallelismo nell'elaborazione.

In [15]:
runnable = RunnableParallel(
    multiply_by_3=RunnablePassthrough.assign(mult_result=lambda x: x["num"] * 3),  # `RunnablePassthrough` porta avanti l'input insieme al risultato
    plus_one_result=lambda x: x["num"] + 1,                                        # e il metodo`assign` permette di aggiungere nuove chiavi all'output
)

runnable.invoke({"num": 5})

{'multiply_by_3': {'num': 5, 'mult_result': 15}, 'plus_one_result': 6}

In [17]:
funny_prompt = ChatPromptTemplate.from_template("crea un proverbio divertente sul tema {topic}")

parallel_chain = (
    prompt
    | model
    | StrOutputParser()
    | RunnableParallel(
        description=RunnablePassthrough(),
        check=(
            analysis_prompt
            | model
            | StrOutputParser()
        ),
        funny=(
            funny_prompt
            | model
            | StrOutputParser()
        )
    )
)

In [18]:
parallel_chain.invoke("sognare")

{'description': 'Sognare è il processo di dormire con ricordi vividi e emozioni forti.',
 'check': 'La tua descrizione non è del tutto corretta.\n\n"Un sogno" (sogno, singolare) è il prodotto dell\'attività cerebrale durante lo stato di sonno, caratterizzato da immagini e sensazioni che possono essere simili a ricordi vividi. Tuttavia, "Sognare" (sognare, verbo) è un verbo che significa provare a sognare o avere un sogno.\n\nPer esempio:\n\n* "Sto sognando una vacanza al mare." (qui si sta utilizzando il verbo "sognare")\n* "Ho fatto un lungo sogno la notte scorsa e ricordo tutti i dettagli." (qui si sta utilizzando l\'aggettivo "lungo" per descrivere lo stato del sogno)\n\nQuindi, la frase corretta sarebbe:\n\n"Sognare è il processo di dormire con ricordi vividi e emozioni forti."\n\nMa, come ti ho detto, anche lì c\'è un piccolo errore. La parola "sognare" non ha senso in quel contesto, quindi la frase corretta sarebbe:\n\n"Dormire è il processo di sognare con ricordi vividi e emozio

<hr>

## DYD
    
...provo a lasciarvi qualche idea su script da creare, sulla falsa riga di questo Notebook, giusto per prendere confidenza con sintassi e framework; sarò quanto più generale possibile per lasciarvi ampio spazio a interpretare e implementare queste (come anche altre) idee:
    
* simulazione interviste con personaggi storici, figure pubbliche o personaggi immaginari, sfruttando i rami paralleli per creare risposte focalizzate su particolari temi, validare quanto generato, estrarre il sentiment o l'argomento principale
* pianificazione eventi, con rami paralleli per logistica, catering, intrattenimento e budget
* gestione di piani di viaggi, con rami paralleli per spostamenti, pernottamenti, vitto, extra, ...
* creazione di curriculum vitae, sfruttando i rami paralleli per la precisa descrizione delle differenti sezioni
* analisi e generazione di recensioni di prodotti o servizi, reali o inventati
* simulazione di una gestione di una crisi aziendale o pubblica, con rami paralleli per comunicati stampa, risposte ai media e azioni immediate da intraprendere, valutando efficacia e coerenza delle strategie proposte
* generazione di personaggi unici e dettagliati per giochi di ruolo, includendo background, abilità speciali e motivazioni personali

🙂

Buon divertimento 👾